<a href="https://colab.research.google.com/github/beatricekiplagat/Deepfake-Audio-Recognition/blob/main/DeepFake_Audio_Detection_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DEEPFAKE AUDIO DETECTION

Workflow
1. Load the dataset 
2. preprocessing - listen to the samples (real and spoofed)
3. convert into spectrogram then to mel spectrogram 
4. choose a suitable sample
5. create a model 
- Pre-Trained Models for Image Classification
VGG-16 - research on all of these
ResNet50
Inceptionv3
EfficientNet
6. Fine tune the model
6. deploy it on website

 

# Importing Libraries

In [ ]:
!pip install wandb
!pip install playsound

In [ ]:
import numpy as np 
import pandas as pd
import os
import glob
import soundfile as sf
import scipy.io.wavfile
import scipy.signal as signal
import matplotlib.pyplot as plt
import pathlib
import shutil
import random
import wandb
from IPython.display import Audio
from scipy.io import wavfile
from playsound import playsound
import librosa
import librosa.display

playsound is relying on another python subprocess. Please use `pip install pygobject` if you want playsound to run more efficiently.


# Load the Data

In [ ]:
# Unzipping the file 
!unzip "/content/drive/MyDrive/projectweeks/LA.zip" 

In [ ]:
# Initialize wandb
wandb.login()
wandb.init(project='DeepFake Audio Detection', entity='ruoro')


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ruoro (use `wandb login --relogin` to force relogin)


In [ ]:
# Checking the working directory 
os.getcwd()
# Changing the Working Directory 
os.chdir("/content/LA")
# Confimation
os.getcwd()

'/content/LA'

In [ ]:
# Create the dataset 
df = pd.read_csv("/content/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.dev.trl.txt", sep = " ", names =  ("a","id","c","fakeclass","class"))
df.head()

,a,id,c,fakeclass,class
0,LA_0069,LA_D_1047731,-,-,bonafide
1,LA_0069,LA_D_1105538,-,-,bonafide
2,LA_0069,LA_D_1125976,-,-,bonafide
3,LA_0069,LA_D_1293230,-,-,bonafide
4,LA_0069,LA_D_1340209,-,-,bonafide


In [ ]:
# Missing Files
df.isna().sum()

a            0
id           0
c            0
fakeclass    0
class        0
dtype: int64

In [ ]:
# Duplicated data
df.duplicated().sum()

0

That means there is a lot of dissimilarity. But we need to separate the data ie the first bonafide and the spoof class


In [ ]:
df1 = df[df['fakeclass']== "-"]
df1.shape

(2548, 5)

In [ ]:
# so now that df1 is only the bonified data we can get 

In [ ]:
# Move all the real audio files in one location
pathlib.Path(f'AudioSample/real').mkdir(parents = True ,exist_ok = True)
real_audio = df1['id']

#
for file in real_audio:
  file_name = os.path.join("/content/LA/ASVspoof2019_LA_dev/flac/"+ file +".flac" )
  if os.path.isfile(file_name):
    # shutil.move("path/to/current/file", "path/to/new/destination/for/file")
    shutil.copy(file_name, '/content/LA/AudioSample/real')


In [ ]:
# Log wandb sample
# sampl

In [ ]:
df2 = df[df['fakeclass'].isin(['A01', 'A02', 'A03', 'A04', 'A07'])]
df2.shape

(14864, 5)

In [ ]:
# Move all the real audio files in one location
pathlib.Path(f'AudioSample/tts').mkdir(parents = True ,exist_ok = True)
tts_audio = df2['id']

#
for file in tts_audio:
  file_name = os.path.join("/content/LA/ASVspoof2019_LA_dev/flac/"+ file +".flac" )
  if os.path.isfile(file_name):
    # shutil.move("path/to/current/file", "path/to/new/destination/for/file")
    shutil.copy(file_name, '/content/LA/AudioSample/tts')

In [ ]:
# log the real audio samples on wandb 
# for loop 

In [ ]:
# Sample . 

In [ ]:
df3 = df[df['fakeclass'].isin(['A05', 'A06', 'A17', 'A18', 'A19'])]
df3.shape

(7432, 5)

In [ ]:
# Move all the Vocoder synthesized audio files in one location
pathlib.Path(f'AudioSample/voc').mkdir(parents = True ,exist_ok = True)
name_of_audio = df3['id']

#
for file in name_of_audio:
  file_name = os.path.join("/content/LA/ASVspoof2019_LA_dev/flac/", file +".flac" )
  if os.path.isfile(file_name):
    shutil.copy(file_name, 'AudioSample/voc')

In [ ]:
# log inti wandb 


In [ ]:
# sample

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# 
# clas = px.df['class'].value_counts()
fig = go.Figure(data=[go.Pie(labels=['Spoof','Bonafide'],
                             values=[2548,22296])])
fig.update_traces(hoverinfo='label+percent', textinfo='value+label', textfont_size=15,
                  marker=dict(line=dict(color='#000000', width=2)))
fig.show()

There is a lot of Data that is spoofed so we will have to segment it. 

In [ ]:
# Finding out the number of classes in the data
df['class'].value_counts()

spoof       22296
bonafide     2548
Name: class, dtype: int64

In [ ]:
fakeclass = df['fakeclass'].value_counts()
fakeclass.to_dict()

{'-': 2548,
 'A01': 3716,
 'A02': 3716,
 'A03': 3716,
 'A04': 3716,
 'A05': 3716,
 'A06': 3716}

In [ ]:
# import plotly.graph_objects as go
# fig = go.Figure(data=[go.Pie(fakeclass)])
#                 # marker=dict(line=dict(color='#000000', width=2)))
# fig.show()

In [ ]:
Audio("/content/LA/AudioSample/real/LA_D_1024892.flac")

In [ ]:
# We should stratify the data into  smaller dataframe that is more managable to handle. 
import random

random.seed(42)

# Separating the bonafide and spoof

bonafide = df[df['class'] == 'bonafide']
spoof = df[df['class']== "spoof"]

# Sampling
bonafide_samp = bonafide.sample(n = 1000)
spoof = spoof.sample (n = 1000)


# Concat.
stratified_df = pd.concat([bonafide_samp, spoof], ignore_index=True)
stratified_df.head()

,a,id,c,fakeclass,class
0,LA_0072,LA_D_8473267,-,-,bonafide
1,LA_0102,LA_D_6919759,-,-,bonafide
2,LA_0076,LA_D_2895802,-,-,bonafide
3,LA_0070,LA_D_6288751,-,-,bonafide
4,LA_0107,LA_D_6838449,-,-,bonafide


In [ ]:
stratified_df['fakeclass'].value_counts()

-      1000
A05     181
A06     175
A03     170
A04     164
A01     156
A02     154
Name: fakeclass, dtype: int64

In [ ]:
stratified_df['class'].value_counts()

spoof       1000
bonafide    1000
Name: class, dtype: int64

Well there are also .....

In [ ]:
Audio("/content/LA/AudioSample/voc/LA_D_1000752.flac")

In [ ]:
print(len(os.listdir('/content/LA/AudioSample/real')))
print(len(os.listdir('/content/LA/AudioSample/voc')))
print(len(os.listdir('/content/LA/AudioSample/tts')))


2548
7432
14864


In [ ]:
# Convert Audio from the DF to to mel specrtogram 
uploaded = .upload()


NameError: ignored

In [ ]:
all_audio = stratified_df['id']
mypath = "/content/LA/ImageData"

pathlib.Path(f'ImageData').mkdir(parents=True, exist_ok=True)
for file in all_audio:
    file_name = os.path.join("/content/LA/ASVspoof2019_LA_dev/flac/"+ file +".flac" )
    y, sy = sf.read(f"{file_name}")
    sy
    # Passing through arguments to the Mel filters
    S = librosa.feature.melspectrogram(y=y, sr=sy, n_mels=128,
                                        fmax=8000)
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(librosa.power_to_db(S,
                                                ref=np.max),
                            y_axis='mel', fmax=8000,
                            x_axis='time')
    plt.colorbar(format='%+2.0f dB')
    plt.tight_layout()
    plt.savefig(f"/content/LA/ImageData/{file}.png")

In [ ]:
# wandb.log({"examples": [wandb.Audio(y, caption="EXA", sample_rate=1600)]})

In [ ]:
len(os.listdir("/content/drive/MyDrive/projectweeks/ImageData"))

2000

In [ ]:
# The data to be used for our Image CNN model is saved under ImageData. 
# We should split the data into training and testing set

from sklearn.model_selection import train_test_split

stratified_df.columns

random.seed(42)
# The label for this cnn model will be the ID and the target will be class
X = stratified_df[['id', 'class']]
y = stratified_df['class']

# train test split
# We will split the data into a 80 - 20 split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = .2, random_state = 42)


In [ ]:

X_train['class'].value_counts()

bonafide    801
spoof       799
Name: class, dtype: int64

In [ ]:
# Transfer the training data into folders
pathlib.Path(f'/content/drive/MyDrive/projectweeks/Train/Bonafide').mkdir(parents = True ,exist_ok = True)

bonafied_fold = X_train[X_train["class"]== 'bonafide']
bona_id = bonafied_fold['id'] 

for file in bona_id:
  file_name = os.path.join("/content/drive/MyDrive/projectweeks/ImageData/"+ file +".png" )
  if os.path.isfile(file_name):
    # shutil.move("path/to/current/file", "path/to/new/destination/for/file")
    shutil.copy(file_name, '/content/drive/MyDrive/projectweeks/Train/Bonafide')

In [ ]:
len(os.listdir('/content/drive/MyDrive/projectweeks/Train/Bonafide'))

800

In [ ]:
# Transfer the training data into folders
#make the folder 
pathlib.Path(f'/content/drive/MyDrive/projectweeks/Train/Spoof').mkdir(parents = True ,exist_ok = True)

# Separate the data 
spoofed_data = X_train[X_train["class"] == 'spoof']
spoof_id = spoofed_data['id'] # get the id

for file in spoof_id:
  file_name = os.path.join("/content/drive/MyDrive/projectweeks/ImageData/"+ file +".png" )
  if os.path.isfile(file_name):
    # shutil.move("path/to/current/file", "path/to/new/destination/for/file")
    shutil.copy(file_name, '/content/drive/MyDrive/projectweeks/Train/Spoof')

In [ ]:
len(os.listdir('/content/drive/MyDrive/projectweeks/Train/Spoof'))

799

In [ ]:
# Now we have to do the same for the test data 
# Transfer the testing data into folders
pathlib.Path(f'/content/drive/MyDrive/projectweeks/Test/Bonafide').mkdir(parents = True ,exist_ok = True)

bonafied_fold = X_test[X_test["class"]== 'bonafide']
bona_id = bonafied_fold['id'] 

for file in bona_id:
  file_name = os.path.join("/content/drive/MyDrive/projectweeks/ImageData/"+ file +".png" )
  if os.path.isfile(file_name):
    # shutil.move("path/to/current/file", "path/to/new/destination/for/file")
    shutil.copy(file_name, '/content/drive/MyDrive/projectweeks/Test/Bonafide')

In [ ]:
len (os.listdir('/content/drive/MyDrive/projectweeks/Test/Bonafide'))

199

In [ ]:
pathlib.Path(f'/content/drive/MyDrive/projectweeks/Test/Spoof').mkdir(parents = True ,exist_ok = True)

# Separate the data 
spoofed_data = X_test[X_test["class"] == 'spoof']
spoof_id = spoofed_data['id'] # get the id

for file in spoof_id:
  file_name = os.path.join("/content/drive/MyDrive/projectweeks/ImageData/"+ file +".png" )
  if os.path.isfile(file_name):
    # shutil.move("path/to/current/file", "path/to/new/destination/for/file")
    shutil.copy(file_name, '/content/drive/MyDrive/projectweeks/Test/Spoof')

In [ ]:
len (os.listdir('/content/drive/MyDrive/projectweeks/Test/Spoof'))

201

In [ ]:
# now all the training and testing data is in the right place. 
# next step is the modeling